# Orders

✏️ **Exercise**

Today, we will investigate the **orders**, and their associated review score.

For that purpose, we will create one single data table containing **all our orders with some engineered statistics for them as additional columns.**

Our goal is to create the following DataFrame, which will come in very handy for our modeling phase

  - `order_id` (_str) the id of the order_
  - `wait_time` (_float) the number of days between order_date and delivered_date_
  - `expected_wait_time` (_float) the number of days between order_date and estimated_delivery_date_
  - `delay_vs_expected` (_float) if the actual delivery date is later than the estimated delivery date, returns the absolute number of days between the two dates, otherwise return 0_
  - `order_status` (_str) the status of the order_
  - `dim_is_five_star` (_int) 1 if the order received a five_star, 0 otherwise_
  - `dim_is_one_star` (_int) 1 if the order received a one_star, 0 otherwise_
  - `review_score`(_int) from 1 to 5_
  - `number_of_product` (_int) number of products that the order contains_
  - `number_of_sellers` (_int) number of sellers involved in the order_
  - `price` (_float) total price of the order paid by customer_
  - `freight_value` (_float) value of the freight paid by customer_
  - (Optional) `distance_customer_seller` (_float) the distance in km between customer and seller_
  
We also want to filter out "non-delivered" orders, unless explicitly specified

❓ **Your challenge**: 

- Implement each feature as a separate method within the `Order` class available at `olist/order.py`
- Then, create a method `get_training_data()` that returns the complete DataFrame.

💡 Suggested methodology:
- Use the notebook below to write and test your code step-by-step first
- Then copy the code into `order.py` once you are certain of your code logic
- Focus on the data manipulation logic now, we will analyse the dataset visually in the next challenges

🔥 Notebook best practices (must-read) 👇

<details>
    <summary> - <i>click here</i> - </summary>

From now on, exploratory notebooks are going to become pretty long, and we strongly advise you to follow these notebook principles:
- Code your logic so that your Notebook can always be ran from top to bottom without crashing (Cell --> Run All)
- Name your variables carefully 
- Use dummy names such as `tmp` or `_` for intermediary steps when you know you won't need them for long
- Clear your code and merge cells when relevant to minimize Notebook size (`Shift-M`)
- Hide your cell output if you don't need to see it anymore (double-click on the red `Out[]:` section to the left of your cell).
- Make heavy use of jupyter nbextention `Collapsable Headings` and `Table of Content` (call a TA if you can't find them)
- Use the following shortcuts 
    - `a` to insert a cell above
    - `b` to insert a cell below
    - `dd` to delete a cell
    - `esc` and `arrows` to move between cells
    - `Shift-Enter` to execute cell and move focus to the next one
    - use `Shift + Tab` when you are between method brackets e.g. `groupby()` to get the docs! Repeat a few times to open it permanently

</details>





In [1]:
# Auto reload imported module everytime a jupyter cell is executed (handy for olist.order.py updates)
%load_ext autoreload
%autoreload 2

In [2]:
# Import usual modules
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [3]:
# Import olist data
from olist.data import Olist
olist = Olist()
data = olist.get_data()
matching_table = olist.get_matching_table()

## Code `order.py`

In [4]:
orders = data['orders'].copy() # good practice to be sure not to modify your `data` variable

### `get_wait_time`
    Return a Dataframe with:
           order_id, wait_time, expected_wait_time, delay_vs_expected, order_status

Hints:
- Don't forget to convert dates from "string" type to "pandas.datetime' using [`pandas.to_datetime()`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.to_datetime.html)
- Take time to understand what python [`datetime`](https://docs.python.org/3/library/datetime.html) objects are 

🎁 We give you the pseudo-code below for this first method:

> 1. Inspect the orders dataframe
2. Filter the dataframe on delivered orders
3. handle datetime
4. compute wait_time
5. compute expected_wait_time
6. compute delay_vs_expected
7. check the new dataframe 
8. once you are satisfied with your code, you can carefully copy-paste it from the notebook to to `olist/order.py`

In [5]:
# YOUR CODE HERE

👀 Check the dataframe you've just created. <br/> 

💪 When your code works, commit it to `olist/order.py` <br/>

🙏 Now, test it by running the following cell 👇 

In [14]:
# Test your code here
from olist.order import Order
Order().get_wait_time()

### `get_review_score`
     Returns a DataFrame with:
        order_id, dim_is_five_star, dim_is_one_star, review_score

In [15]:
reviews = data['order_reviews'].copy()
reviews

👉 Let's create two functions `dim_five_star` and `dim_one_star`  
    We will apply them  `element_wise` to the `review_score` column in the next cell below.

In [16]:
def dim_five_star(d):
    pass  # YOUR CODE HERE


def dim_one_star(d):
    pass  # YOUR CODE HERE

👉 Use these functions to create two boolean features `dim_is_five_star` and `dim_is_one_star`

In [17]:
# YOUR CODE HERE

Once again, 

👀 Check the dataframe you've just created. <br/> 

💪 When your code works, commit it to `olist/order.py` <br/>

🙏 Now, test it by running the following cell 👇 

In [19]:
# Test your code here
from olist.order import Order
Order().get_review_score()

#### Check your code

In [20]:
from nbresult import ChallengeResult

result = ChallengeResult('reviews',
    dim_five_star=dim_five_star(5),
    dim_not_five_star=dim_five_star(3),
    dim_one_star=dim_one_star(1),
    dim_not_one_star=dim_one_star(2)
)
result.write()
print(result.check())

### `get_number_products`:
     Returns a DataFrame with:
        order_id, number_of_products (total number of products per order)

In [21]:
# YOUR CODE HERE

👉 Same routine: 
* check your dataframe, 
* commit your code to `olist/order.py`
* and check that it truly works.

In [22]:
# Check your new dataframe and commit your code to olist/order.py when it works. 
Order().get_number_products()

### `get_number_sellers`:
     Returns a DataFrame with:
        order_id, number_of_sellers (total number of unique sellers per order)

<details>
    <summary>- <i>Hint</i> -</summary>

`pd.Series.nunique()`
</details>

In [23]:
# YOUR CODE HERE

In [24]:
# Check your new dataframe and commit your code to olist/order.py when it works. 
Order().get_number_sellers()

### `get_price_and_freight`
     Returns a DataFrame with:
        order_id, price, freight_value

<details>
    <summary>- <i>Hint</i></summary>

`pd.Series.agg()` allows you to apply one transformation method per column of your groupby object
</details>

In [25]:
# YOUR CODE HERE

In [26]:
# Check your new dataframe and commit your code to olist/order.py when it works. 
Order().get_price_and_freight()

### `get_distance_seller_customer` 
**(OPTIONAL - Try  to code this function only after finishing today's challenges - Skip to next section)**

    Returns a Dataframe with:
        [order_id, distance_seller_customer] 
               (the distance in km between customer and seller)

💡Have a look at the `haversine_distance` formula we coded for you in the `olist.utils` module

In [27]:
# YOUR CODE HERE

In [47]:
# Check your new dataframe and commit your code to olist/order.py when it works. 
Order().get_distance_seller_customer()

# Test your newly coded module

❓ Time to code `get_training_data` making use of your previous coded methods.

In [49]:
from olist.order import Order
from nbresult import ChallengeResult
data = Order().get_training_data()
result = ChallengeResult('training',
    shape=data.shape,
    columns=sorted(list(data.columns))
)
result.write()
print(result.check())

🏁 Congratulations! 

⌛️ Commit and push your notebook before starting the next challenge.